In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler
from sklearn import neighbors
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import recall_score, make_scorer, accuracy_score, precision_score,classification_report, confusion_matrix, roc_curve, roc_auc_score,auc
from pandas import set_option


In [2]:
set_option('display.max_row', 10)
set_option('display.width', 100)
set_option('precision', 2)
set_option('display.max_column', 40)

In [3]:
df_korban = pd.read_csv('datafraud.csv') 
df_korban_fraud = df_korban[df_korban.LABEL==1].copy()
df_non_korban = pd.read_csv('datanonfraud.csv')
frames= [df_korban_fraud, df_non_korban.sample(8821)] # SAMPEL TERDIRI DARI 10000 BARIS YANG DIAMBIL SECARA ACAK
data= pd.concat(frames)
y_true = data['LABEL']
data.head()
y_true=data['LABEL']

C:\Users\leno\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data[data['LABEL']==1].describe()    # Describing the data

,LABEL,fiturid,avlblnc,txamount,transtime,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo
count,1179.0,1.18e+03,1.18e+03,1.18e+03,1179.00,1179.00,1179.00,1179.00,1.18e+03,1.18e+03,1.18e+03,1.18e+03
mean,1.0,1.13e+08,2.23e+07,2.44e+06,42452.84,3.21,0.49,64.25,3.45e+04,7.54e+05,3.45e+04,4.84e+01
std,0.0,3.04e+07,1.89e+08,9.12e+06,20143.72,3.27,0.50,626.61,6.83e+05,1.56e+07,6.83e+05,1.49e+03
min,1.0,1.03e+08,0.00e+00,1.00e+03,8.00,1.00,0.00,0.00,0.00e+00,0.00e+00,0.00e+00,4.08e-05
25%,1.0,1.03e+08,0.00e+00,4.00e+05,27799.00,1.00,0.00,0.00,0.00e+00,0.00e+00,0.00e+00,9.62e-02
50%,1.0,1.03e+08,7.23e+05,1.25e+06,41328.00,2.00,0.00,25.00,6.13e-01,9.09e-01,3.33e-01,3.84e-01
75%,1.0,1.03e+08,6.56e+06,2.00e+06,59873.00,4.00,1.00,34.00,2.04e+00,2.53e+00,9.50e-01,9.04e-01
max,1.0,2.06e+08,2.87e+09,2.65e+08,85168.00,32.00,1.00,16804.00,1.50e+07,5.23e+08,1.50e+07,5.10e+04


In [5]:
data.shape

(10000, 19)

In [6]:
data.head()

,LABEL,fiturid,responcode,avlblnc,txamount,transtime,terminaltype,jenis_kartu,groupname,dana_keluar,lokasi,responcode_before,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo
3853,1,103010020,0,1.15e+06,100000,28893,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,1,0,0.0,0.0,0.86,0.08
8023,1,103010020,51,0.00e+00,200000,4099,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,0,0,0.0,0.0,0.60,3.83
8024,1,103010020,0,5.23e+04,150000,4123,ATM_BRI,BRITAMA,TarikTunai,True,dalam,51,2,0,24,0.0,0.0,0.25,6.70
9960,1,103010020,0,5.00e+05,1000000,48153,ATM_BRI,SIMPEDES,TarikTunai,True,dalam,0,1,0,0,0.0,0.0,0.33,2.00
10485,1,103010020,0,4.59e+07,1000000,69395,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,0,0,0.0,0.0,0.60,0.02


In [7]:
data.duplicated().sum()

4

In [8]:
data.loc[data.duplicated(keep = False), :] 

,LABEL,fiturid,responcode,avlblnc,txamount,transtime,terminaltype,jenis_kartu,groupname,dana_keluar,lokasi,responcode_before,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo
270710,0,201010170,0,7.33e+04,0,81099,INTERNET BANKING,SIMPEDES,CekSaldo,False,dalam,0,0,1,5579,0.0,0.0,0.0,0.0
271147,0,201010170,0,7.91e+04,0,26271,INTERNET BANKING,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,0.0,0.0
271168,0,201010170,0,7.91e+04,0,26271,INTERNET BANKING,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,0.0,0.0
270684,0,201010170,0,7.33e+04,0,81099,INTERNET BANKING,SIMPEDES,CekSaldo,False,dalam,0,0,1,5579,0.0,0.0,0.0,0.0
227075,0,201010170,0,2.96e+06,0,39357,SMS BANKING,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,0.0,0.0
268849,0,201010170,0,1.73e+07,0,37936,INTERNET BANKING,BRITAMA,CekSaldo,False,dalam,0,0,1,40,0.0,0.0,0.0,0.0
268801,0,201010170,0,1.73e+07,0,37936,INTERNET BANKING,BRITAMA,CekSaldo,False,dalam,0,0,1,40,0.0,0.0,0.0,0.0
228056,0,201010170,0,2.96e+06,0,39357,SMS BANKING,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,0.0,0.0


In [9]:
data.drop_duplicates(keep= 'first').shape

(9996, 19)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 3853 to 259348
Data columns (total 19 columns):
LABEL                10000 non-null int64
fiturid              10000 non-null int64
responcode           10000 non-null object
avlblnc              10000 non-null float64
txamount             10000 non-null int64
transtime            10000 non-null int64
terminaltype         10000 non-null object
jenis_kartu          10000 non-null object
groupname            10000 non-null object
dana_keluar          10000 non-null bool
lokasi               10000 non-null object
responcode_before    10000 non-null object
seq.trxday           10000 non-null int64
CekSaldo.before      10000 non-null int64
diff.time            10000 non-null int64
diff.txamount        10000 non-null float64
diff.txamount.trx    10000 non-null float64
diff.txamount.bef    10000 non-null float64
percent_saldo        10000 non-null float64
dtypes: bool(1), float64(5), int64(7), object(6)
memory usage: 1.5+ MB


In [11]:
data.responcode =pd.to_numeric(data.responcode, errors='coerce').fillna(0, downcast='infer') # TERDAPAT NAN YANG HARUS DIHILANGKAN
data.responcode_before =pd.to_numeric(data.responcode_before, errors='coerce').fillna(0, downcast='infer')

In [12]:
data[data['LABEL']==1].describe()  

,LABEL,fiturid,responcode,avlblnc,txamount,transtime,responcode_before,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo
count,1179.0,1.18e+03,1179.00,1.18e+03,1.18e+03,1179.00,1179.0,1179.00,1179.00,1179.00,1.18e+03,1.18e+03,1.18e+03,1.18e+03
mean,1.0,1.13e+08,14.80,2.23e+07,2.44e+06,42452.84,12.0,3.21,0.49,64.25,3.45e+04,7.54e+05,3.45e+04,4.84e+01
std,0.0,3.04e+07,25.06,1.89e+08,9.12e+06,20143.72,23.2,3.27,0.50,626.61,6.83e+05,1.56e+07,6.83e+05,1.49e+03
min,1.0,1.03e+08,0.00,0.00e+00,1.00e+03,8.00,0.0,1.00,0.00,0.00,0.00e+00,0.00e+00,0.00e+00,4.08e-05
25%,1.0,1.03e+08,0.00,0.00e+00,4.00e+05,27799.00,0.0,1.00,0.00,0.00,0.00e+00,0.00e+00,0.00e+00,9.62e-02
50%,1.0,1.03e+08,0.00,7.23e+05,1.25e+06,41328.00,0.0,2.00,0.00,25.00,6.13e-01,9.09e-01,3.33e-01,3.84e-01
75%,1.0,1.03e+08,51.00,6.56e+06,2.00e+06,59873.00,0.0,4.00,1.00,34.00,2.04e+00,2.53e+00,9.50e-01,9.04e-01
max,1.0,2.06e+08,93.00,2.87e+09,2.65e+08,85168.00,61.0,32.00,1.00,16804.00,1.50e+07,5.23e+08,1.50e+07,5.10e+04


In [13]:
data= data.dropna()

In [14]:
one_hot = pd.get_dummies(data['responcode']) # RESPONCODE MEMILIKI TYPE OBJECT SEHINGGA HARUS DILAKUKAN ONE HOT ENCODING 
data= data.drop('responcode', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,terminaltype,jenis_kartu,groupname,dana_keluar,lokasi,responcode_before,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,92,93
3853,1,103010020,1.15e+06,100000,28893,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,0,0
8023,1,103010020,0.00e+00,200000,4099,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,0,0,0.00,0.00,0.60,3.83,0,0,1,0,0,0,0
8024,1,103010020,5.23e+04,150000,4123,ATM_BRI,BRITAMA,TarikTunai,True,dalam,51,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,0,0
9960,1,103010020,5.00e+05,1000000,48153,ATM_BRI,SIMPEDES,TarikTunai,True,dalam,0,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,0,0
10485,1,103010020,4.59e+07,1000000,69395,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192077,0,205010360,0.00e+00,300000,30138,INTERNET BANKING,BRITAMA,TransferBRI,True,dalam,0,29,1,152,7.47,7.47,0.40,1.82,1,0,0,0,0,0,0
227456,0,201010170,1.59e+06,0,77911,INTERNET BANKING,BRITAMA,CekSaldo,False,dalam,0,0,1,2,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0
131926,0,201010170,2.88e+06,0,65516,ATM_BRI,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0
164284,0,205010360,0.00e+00,50000,65278,INTERNET BANKING,BRITAMA,TransferBRI,True,dalam,0,197,1,45,163.01,163.01,0.98,1.50,1,0,0,0,0,0,0


In [15]:
one_hot = pd.get_dummies(data['responcode_before'])# RESPONCODE_BEFORE MEMILIKI TYPE OBJECT SEHINGGA HARUS DILAKUKAN ONE HOT ENCODING 
data= data.drop('responcode_before', axis =1)
data= pd.concat([data, one_hot], axis=1)


In [16]:
data.head()

,LABEL,fiturid,avlblnc,txamount,transtime,terminaltype,jenis_kartu,groupname,dana_keluar,lokasi,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,92,93,0,3,30,51,61,92
3853,1,103010020,1.15e+06,100000,28893,ATM_BRI,BRITAMA,TarikTunai,True,dalam,1,1,0,0.0,0.0,0.86,0.08,1,0,0,0,0,0,0,1,0,0,0,0,0
8023,1,103010020,0.00e+00,200000,4099,ATM_BRI,BRITAMA,TarikTunai,True,dalam,1,0,0,0.0,0.0,0.60,3.83,0,0,1,0,0,0,0,1,0,0,0,0,0
8024,1,103010020,5.23e+04,150000,4123,ATM_BRI,BRITAMA,TarikTunai,True,dalam,2,0,24,0.0,0.0,0.25,6.70,1,0,0,0,0,0,0,0,0,0,1,0,0
9960,1,103010020,5.00e+05,1000000,48153,ATM_BRI,SIMPEDES,TarikTunai,True,dalam,1,0,0,0.0,0.0,0.33,2.00,1,0,0,0,0,0,0,1,0,0,0,0,0
10485,1,103010020,4.59e+07,1000000,69395,ATM_BRI,BRITAMA,TarikTunai,True,dalam,1,0,0,0.0,0.0,0.60,0.02,1,0,0,0,0,0,0,1,0,0,0,0,0


In [17]:
one_hot = pd.get_dummies(data['terminaltype'])
data= data.drop('terminaltype', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,jenis_kartu,groupname,dana_keluar,lokasi,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,92,93,0,3,30,51,61,92,ATM MERAH PUTIH,ATM_BRI,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING
3853,1,103010020,1.15e+06,100000,28893,BRITAMA,TarikTunai,True,dalam,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
8023,1,103010020,0.00e+00,200000,4099,BRITAMA,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.60,3.83,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
8024,1,103010020,5.23e+04,150000,4123,BRITAMA,TarikTunai,True,dalam,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
9960,1,103010020,5.00e+05,1000000,48153,SIMPEDES,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
10485,1,103010020,4.59e+07,1000000,69395,BRITAMA,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192077,0,205010360,0.00e+00,300000,30138,BRITAMA,TransferBRI,True,dalam,29,1,152,7.47,7.47,0.40,1.82,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
227456,0,201010170,1.59e+06,0,77911,BRITAMA,CekSaldo,False,dalam,0,1,2,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
131926,0,201010170,2.88e+06,0,65516,BRITAMA,CekSaldo,False,dalam,0,1,0,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
164284,0,205010360,0.00e+00,50000,65278,BRITAMA,TransferBRI,True,dalam,197,1,45,163.01,163.01,0.98,1.50,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [18]:
one_hot = pd.get_dummies(data['jenis_kartu'])
data= data.drop('jenis_kartu', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,groupname,dana_keluar,lokasi,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,...,93,0,3,30,51,61,92,ATM MERAH PUTIH,ATM_BRI,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES
3853,1,103010020,1.15e+06,100000,28893,TarikTunai,True,dalam,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
8023,1,103010020,0.00e+00,200000,4099,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.60,3.83,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
8024,1,103010020,5.23e+04,150000,4123,TarikTunai,True,dalam,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
9960,1,103010020,5.00e+05,1000000,48153,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
10485,1,103010020,4.59e+07,1000000,69395,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192077,0,205010360,0.00e+00,300000,30138,TransferBRI,True,dalam,29,1,152,7.47,7.47,0.40,1.82,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
227456,0,201010170,1.59e+06,0,77911,CekSaldo,False,dalam,0,1,2,0.00,0.00,0.00,0.00,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
131926,0,201010170,2.88e+06,0,65516,CekSaldo,False,dalam,0,1,0,0.00,0.00,0.00,0.00,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
164284,0,205010360,0.00e+00,50000,65278,TransferBRI,True,dalam,197,1,45,163.01,163.01,0.98,1.50,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [19]:
one_hot = pd.get_dummies(data['groupname'])
data= data.drop('groupname', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,dana_keluar,lokasi,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,92,...,61,92,ATM MERAH PUTIH,ATM_BRI,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES,CekSaldo,TarikTunai,TransferBRI,TransferBRIVA,TransferBersama
3853,1,103010020,1.15e+06,100000,28893,True,dalam,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
8023,1,103010020,0.00e+00,200000,4099,True,dalam,1,0,0,0.00,0.00,0.60,3.83,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
8024,1,103010020,5.23e+04,150000,4123,True,dalam,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
9960,1,103010020,5.00e+05,1000000,48153,True,dalam,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
10485,1,103010020,4.59e+07,1000000,69395,True,dalam,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192077,0,205010360,0.00e+00,300000,30138,True,dalam,29,1,152,7.47,7.47,0.40,1.82,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0
227456,0,201010170,1.59e+06,0,77911,False,dalam,0,1,2,0.00,0.00,0.00,0.00,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0
131926,0,201010170,2.88e+06,0,65516,False,dalam,0,1,0,0.00,0.00,0.00,0.00,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
164284,0,205010360,0.00e+00,50000,65278,True,dalam,197,1,45,163.01,163.01,0.98,1.50,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0


In [20]:
one_hot = pd.get_dummies(data['dana_keluar'])
data= data.drop('dana_keluar', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,lokasi,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,92,93,...,ATM MERAH PUTIH,ATM_BRI,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES,CekSaldo,TarikTunai,TransferBRI,TransferBRIVA,TransferBersama,False,True
3853,1,103010020,1.15e+06,100000,28893,dalam,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
8023,1,103010020,0.00e+00,200000,4099,dalam,1,0,0,0.00,0.00,0.60,3.83,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
8024,1,103010020,5.23e+04,150000,4123,dalam,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
9960,1,103010020,5.00e+05,1000000,48153,dalam,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1
10485,1,103010020,4.59e+07,1000000,69395,dalam,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192077,0,205010360,0.00e+00,300000,30138,dalam,29,1,152,7.47,7.47,0.40,1.82,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1
227456,0,201010170,1.59e+06,0,77911,dalam,0,1,2,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0
131926,0,201010170,2.88e+06,0,65516,dalam,0,1,0,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0
164284,0,205010360,0.00e+00,50000,65278,dalam,197,1,45,163.01,163.01,0.98,1.50,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1


In [21]:
one_hot = pd.get_dummies(data['lokasi'])
data= data.drop('lokasi', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,92,93,0,...,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES,CekSaldo,TarikTunai,TransferBRI,TransferBRIVA,TransferBersama,False,True,dalam,luar
3853,1,103010020,1.15e+06,100000,28893,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
8023,1,103010020,0.00e+00,200000,4099,1,0,0,0.00,0.00,0.60,3.83,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
8024,1,103010020,5.23e+04,150000,4123,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
9960,1,103010020,5.00e+05,1000000,48153,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0
10485,1,103010020,4.59e+07,1000000,69395,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192077,0,205010360,0.00e+00,300000,30138,29,1,152,7.47,7.47,0.40,1.82,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0
227456,0,201010170,1.59e+06,0,77911,0,1,2,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0
131926,0,201010170,2.88e+06,0,65516,0,1,0,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0
164284,0,205010360,0.00e+00,50000,65278,197,1,45,163.01,163.01,0.98,1.50,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0


In [22]:
X = data.iloc[:, data.columns != "LABEL"]
Y = data.iloc[:, data.columns == "LABEL"]

In [23]:
X.head()


,fiturid,avlblnc,txamount,transtime,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,92,93,0,3,...,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES,CekSaldo,TarikTunai,TransferBRI,TransferBRIVA,TransferBersama,False,True,dalam,luar
3853,103010020,1.15e+06,100000,28893,1,1,0,0.0,0.0,0.86,0.08,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
8023,103010020,0.00e+00,200000,4099,1,0,0,0.0,0.0,0.60,3.83,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
8024,103010020,5.23e+04,150000,4123,2,0,24,0.0,0.0,0.25,6.70,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
9960,103010020,5.00e+05,1000000,48153,1,0,0,0.0,0.0,0.33,2.00,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0
10485,103010020,4.59e+07,1000000,69395,1,0,0,0.0,0.0,0.60,0.02,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0


In [24]:
from sklearn.model_selection import train_test_split,GridSearchCV, StratifiedKFold
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.37, random_state= None) # UKURAN DATA TESTING YAKNI 2000 BARIS

In [25]:
# STANDARDISASI NILAI PADA SETIAP FITUR
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

In [26]:
print(X_train_std)

[[ 0.49128806 -0.08488063  0.1248117  ...  0.68599434  0.16194372
  -0.16194372]
 [ 0.38516927  0.1408469  -0.19207336 ... -1.45773797  0.16194372
  -0.16194372]
 [ 0.49128806 -0.08488063 -0.15479276 ...  0.68599434  0.16194372
  -0.16194372]
 ...
 [-2.21462112 -0.08373739  0.04093036 ...  0.68599434  0.16194372
  -0.16194372]
 [ 0.49128806 -0.08488063 -0.15479276 ...  0.68599434  0.16194372
  -0.16194372]
 [ 0.49128806 -0.08488063 -0.17343306 ...  0.68599434  0.16194372
  -0.16194372]]


In [27]:
# FITUR ID SUDAH DIWALIKI OLEH GROUPNAME
data.drop(['fiturid', 'dalam', 'luar', 'false', 'true'], axis=1, inplace=True)
data.head()

,LABEL,avlblnc,txamount,transtime,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,92,93,0,3,...,ATM MERAH PUTIH,ATM_BRI,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES,CekSaldo,TarikTunai,TransferBRI,TransferBRIVA,TransferBersama,False,True
3853,1,1.15e+06,100000,28893,1,1,0,0.0,0.0,0.86,0.08,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
8023,1,0.00e+00,200000,4099,1,0,0,0.0,0.0,0.60,3.83,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
8024,1,5.23e+04,150000,4123,2,0,24,0.0,0.0,0.25,6.70,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
9960,1,5.00e+05,1000000,48153,1,0,0,0.0,0.0,0.33,2.00,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1
10485,1,4.59e+07,1000000,69395,1,0,0,0.0,0.0,0.60,0.02,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1


In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint

clf = KNeighborsClassifier(n_jobs=-1)

params = {'n_neighbors':list(range(1,10)),
          'leaf_size':list(range(1,10)),
          'weights':['uniform', 'distance'],
          'algorithm':['auto', 'ball_tree','kd_tree','brute'],
          'n_jobs':[-1]}
#Making models with hyper parameters sets
cv = GridSearchCV(clf, param_grid=params, n_jobs=1)
#Learning
cv.fit(X_train,y_train.values.reshape(-1,))
#The best hyper parameters set
print("Best Hyper Parameters:\n",cv.best_params_)
#Prediction
pred=cv.predict(X_test)
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Accuracy:",metrics.accuracy_score(pred,y_test))
#evaluation(Confusion Metrix)
print("Confusion Metrix:\n",metrics.confusion_matrix(pred,y_test))

Best Hyper Parameters:
 {'algorithm': 'auto', 'leaf_size': 1, 'n_jobs': -1, 'n_neighbors': 9, 'weights': 'distance'}
Accuracy: 0.9581081081081081
Confusion Metrix:
 [[3204   71]
 [  84  341]]
